### RAG
检索增强生成(Retrieval-Augmented Generation, RAG)和Agent应该是目前LLM最火的两个方向，当然这两个方向也不是完全独立的，他们各自能解决各自的问题。我们先来聊聊RAG，引申出langchain中和RAG相关的六个组件。

先从一个问题开始：假设我们有一些特别的知识，比如我们室组内部有一个上线的流程规范，这个流程规范可能在文档中以文字，表格，甚至图片的形式存在着的。但是如果我去问一个模型，和这个流程规范相关的任何问题，它要么告诉你不知道，要么就胡说八道，不可能给我们正确的答案。这时候我们可以构造如下的提示词给到模型，那么凭借语言模型的强大泛化能力就可以回答问题：
```
请根据以下文本回答反引号`中的问题:
XXXXX(这里是室组上线流程的具体规范文字)
`室组上线之前需要经过哪些步骤，哪些需要通过组长审核`
```
这其实就是RAG的核心。但是RAG虽然看起来简单，其实细节非常多，虽然我们上面看所谓RAG不过就是把知识当做提示词一部分给到模型然后提问吗。但是我们需要考虑的问题有很多：
1. 知识是以各种形式存在的，有些是文档，有些是表格，有些是图片，甚至有些是网页，那么如果给这些知识一个统一的抽象
2. 统一抽象的这些知识以什么形式存在哪里
3. 我们知道模型窗口是有上下文大小限制的，那么我们就不可能说把所有的知识全部塞给模型，这样既不高效，也不现实，因此我们只能从里面选择一部分和问题相关的片段出来。那么我们如何将知识划分片段又如何去选择哪些片段是和问题相关的
4. 得到相关的片段之后，我们又该如何组合这些文档片段，成为一个完整的文字交给模型

这些问题其实就是langchain和RAG相关模块帮我们做的：Document Loader，Text Splitters，Embedding Models，Vector Stores和Retrievers。我们来一个个看看。
#### Document Loader
首先langchain将所有支持抽象成了Document对象，里面是一个带有内容的包装类，同时langchain提供了各种各样的加载器，我们不可能一个个全部过一遍，所以你可以[自己去看看](https://python.langchain.com/v0.1/docs/integrations/document_loaders/)，我们这里介绍一个稍微简单点的Loader:WebBaseLoader。
假设我们的问题就是询问和langchain相关的问题，我们先来看看直接询问会是什么样的结果

In [ ]:
from libs.llm.qwen import qwen
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template('什么是langchain')
])
chain = prompt_template | qwen | StrOutputParser()
chain.invoke({})

可以看出，对于模型来说，它并不懂什么是langchain，这可能是因为模型训练的时候langchain还不存在，或者没有使用到任何和langchain相关的数据去进行训练。我们通过RAG的手段给它一段外挂知识来将它一步步优化。首先就是langchain的知识去哪里找，当然是langchain的官方文档，也就是我们需要一个能直接从网页读取数据的loader，而WebBaseLoader就是这么一个Loader。

In [ ]:
# WebBaseLoader.ts
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_path='https://python.langchain.com/v0.2/docs/concepts/',
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_="docItemContainer_Djhp"
    )),
    proxies={'http': 'http://localhost:7890', 'https': 'http://localhost:7890'}
)
doc = loader.load()
print(doc)

在文档完成加载后我们需要对文档进行切分，这时候就需要text splitter
#### Text Splitters
前面已经解释过为啥要对文档进行切分了，但是很明显，对于文档切分应该会有很多策略，不同策略会有不同的效果。langchain的Text Splitters模块就提供了各种切分文档的策略。分割的策略对RAG来说非常重要，总的来说我们要降数据块变小的同时还要让数据块的语义相关，这两者其实本来就很矛盾。比如我有个句子"William曾经做过两年python工程师，他后来又干了7年前端工程师"。如果句子被切割为"William曾经做过两年python工程师"和"他后来又干了7年前端工程师"，那么你在提问"William都干过些什么工作"的时候，可能就只能找到第一条信息，得出他干过两年python的结论。而第二条信息因为他的指代，很可能在向量召回的时候并不能被召回。所以文档的分割需要根据应用场景进行权衡考虑。在langchain中提供了很多的Text Splitters，其中一些很直观，也很简单，比如ChatTextSplitter可以通过你提供的一个字符串(比如\n\t这种)，对文档进行分割。又或者NLTKTextSplitter和SpacyTextSplitter这种，可以以句子为单位进行分割。这些分割方法在合适的场景都有合适的用处，但是他们没有考虑语义，没有考虑前后两个块是否有语义上的关联，是否需要合并为一个chunk，因此langchain又实现了一个语义分割SemanticChunker，文档分割方法可以[参考文档](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)。我们这里以RecursiveCharacterTextSplitter为例来进行演示

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=10,
    separators=['.', ' ', '\n']
)
print(splitter.split_text(doc[0].page_content))

其中chunk_size表示分割后块的大小的最大值，分割后块大小不会超过这个值。chunk_overlap指的是两个相邻文档块的最大重合字符数。比如之前的例子：

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=40,
    chunk_overlap=15,
    separators=['。']
)
splitter.split_text('William曾经做过两年python工程师。他后来又干了七年前端工程师。现在他学起了java。')

#### Embedding和Vector Store
Embedding之前在分享Wolfram的《这就是ChatGPT》的时候就详细介绍过了，因此这里只是简单说下，Embedding模型就是一种向量化的算法模型，它向量化的对象可以是文档，可以是音频，视频，它可以提取对象的特定特征，虽然并不能解释它的特征具体是啥，但是它具有相似的对象在向量上也有更近相似度的特点。embedding模型的选型可以参考Huggingface的[LeaderBoard](https://huggingface.co/spaces/mteb/leaderboard)，我们这里演示为了方便采用的是通义千问的embedding模型text-embedding-v2。

In [ ]:
from langchain_community.embeddings import DashScopeEmbeddings

embeddings = DashScopeEmbeddings(
    model='text-embedding-v2',
    dashscope_api_key='sk-90cba3edc3e1412b84547915475dca30'
)
v = embeddings.embed_query("William曾经做过两年python工程师。他后来又干了七年前端工程师。现在他学起了java。")
len(v)

关于向量数据库，目前也有很多选择，这次我们使用的是Pinecone，因为它是我接触的第一个向量数据库，是以服务方式提供，有免费资源，也有UI界面可以清楚看到结果，比较适合初学。在上手之后可以去学习一些开源的离线部署的向量库，比如milvus，Lancedb，chroma这种。langchain也支持了市面上几乎所有的向量库的连接，具体库的连接就[参考文档](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [ ]:
import bs4
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
import getpass

if os.environ.get('PINECONE_API_KEY') is None:
    os.environ['PINECONE_API_KEY'] = getpass.getpass('PINECONE_API_KEY: ')

index_name = 'share-langchain-rag'

loader = WebBaseLoader(
    web_path='https://www.gov.cn/flfg/2013-02/08/content_2332395.htm',
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_="p1"
    ))
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
embeddings = DashScopeEmbeddings(
    model='text-embedding-v2',
    dashscope_api_key='sk-90cba3edc3e1412b84547915475dca30'
)
docsearch = PineconeVectorStore.from_documents(
    docs,
    embeddings,
    index_name=index_name
)

In [ ]:
docsearch.similarity_search("根据我国著作权保护法，软件保护期是多久", k=10)

上面已经实现了从文档数据分片，然后向量化，然后存放到向量库，最后通过问题相识度搜索召回相关文档片段的过程了。这部分基本就是RAG的离线部分。在完成这些之后，用户就会询问问题，然后将问题也向量化，检索出相关文档，将检索出来的文档给模型然后回答问题，就是RAG的在线部分。而这部分就需要用到Retriever了。
#### Retriever
前面已经通过相似度搜索召回了k个和问题相关的片段了，但是真实的情况远不止这么简单。我们要考虑如何召回相关度最高的k个文档，又或者召回和问题关联度超过一定值的文档。召回的文档如果太多了，全部给模型可能会导致超过窗口大小，那么又如何处理。langchain内置了非常多的Retriever用于解决数据召回，我们这次因为只是总览一下langchain，只介绍其中最简单和常用的一个VectorStoreRetriever，其他可以查阅[文档](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/)

In [ ]:
from langchain_pinecone import PineconeVectorStore

index_name = 'share-langchain-rag'
embeddings = DashScopeEmbeddings(
    model='text-embedding-v2',
    dashscope_api_key='sk-90cba3edc3e1412b84547915475dca30'
)
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
# retriever = vector_store.as_retriever(search_type='similarity_score_threshold',search_kwargs={'k': 3, 'score_threshold': 0.8})
# retriever = vector_store.as_retriever(search_type='mmr',search_kwargs={'k': 3})
retriever.invoke("根据我国著作权保护法，软件保护期是多久")

现在我们基本上完成了RAG需要的所有步骤了，我们实现一般会将离线和在线部分分离开，因为离线部分主要是完成数据的准备，用户侧不会调用，而且一般也是非实时的。而在线部分基本都是用户的访问插叙，一般也要求延迟不要太低。我们将目前的在线部分全部实现出来就是：

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import DashScopeEmbeddings
from libs.llm.ollama import ollama
import getpass
import os

if os.environ.get('PINECONE_API_KEY') is None:
    os.environ['PINECONE_API_KEY'] = getpass.getpass('PINECONE_API_KEY: ')

template = """
你是一个用于问答任务的助手。
使用以下检索到的内容来回答问题。
如果你不知道答案，就说你不知道。
保持答案的简洁
问题: 
-------
{question}
------- 

上下文: 
-------
{context} 
-------
回答:
"""
prompt = ChatPromptTemplate.from_template(template)

index_name = 'share-langchain-rag'
embeddings = DashScopeEmbeddings(
    model='text-embedding-v2',
    dashscope_api_key='sk-90cba3edc3e1412b84547915475dca30'
)
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vector_store.as_retriever(search_type='similarity_score_threshold',
                                      search_kwargs={'k': 3, 'score_threshold': 0.7})
rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | ollama
        | StrOutputParser()
)
rag_chain.invoke("根据我国著作权保护法，软件保护期是多久")